# Import Data

In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

In [8]:
titanic = pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/TheTitanic.csv')
titanic.head()

,1,2,3,4,5,6,7,8,9,10
0,sn,pclass,survived,NaN,gender,age,family,fare,embarked,date
1,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
2,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
3,2,3,0,Master. Eugene Joseph,male,?,2,20.25,S,02-Jan-90
4,3,2,0,"Abbott, Mr. Rossmore Edward",NaN,NaN,2,**,S,03-Jan-90


## Data Preprocessing 

### Create header 

In [19]:
titanic.columns = titanic.iloc[0]
titanic.head()

,sn,pclass,survived,NaN,gender,age,family,fare,embarked,date
0,sn,pclass,survived,NaN,gender,age,family,fare,embarked,date
1,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
2,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
3,2,3,0,Master. Eugene Joseph,male,?,2,20.25,S,02-Jan-90
4,3,2,0,"Abbott, Mr. Rossmore Edward",NaN,NaN,2,**,S,03-Jan-90


In [21]:
titanic = titanic[1:].reset_index(drop=True)
titanic.head()

,sn,pclass,survived,NaN,gender,age,family,fare,embarked,date
0,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
1,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
2,2,3,0,Master. Eugene Joseph,male,?,2,20.25,S,02-Jan-90
3,3,2,0,"Abbott, Mr. Rossmore Edward",NaN,NaN,2,**,S,03-Jan-90
4,4,3,1,"Abbott, Mr. Rossmore Edward",female,35,2,20.25,S,04-Jan-90


In [49]:
titanic.rename(columns={titanic.columns[3]:'name'}, inplace=True)
titanic.head()

,sn,pclass,survived,name,gender,age,family,fare,embarked,date
0,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
1,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
2,2,3,0,Master. Eugene Joseph,male,?,2,20.25,S,02-Jan-90
3,3,2,0,"Abbott, Mr. Rossmore Edward",NaN,NaN,2,**,S,03-Jan-90
4,4,3,1,"Abbott, Mr. Rossmore Edward",female,35,2,20.25,S,04-Jan-90


### Split column name

In [59]:
titanic[['first_person', 'second_person']] = titanic['name

name
Mr. Anthony                    2
Abbott, Mr. Rossmore Edward    2
Connolly, Miss. Kate           2
Kelly, Mr. James               2
Newell, Mr. Arthur Webster     1
                              ..
Garfirth, Mr. John             1
Gallagher, Mr. Martin          1
Gale, Mr. Shadrach             1
Gale, Mr. Harry                1
Zimmerman, Mr. Leo             1
Name: count, Length: 1297, dtype: int64